In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：

bounds = [(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10),(0,10)]
x_opt = [1.66991341326291344e-17, 3.95378229282456509e-16, 3.94599045143233784, 1.06036597479721211e-16, 3.2831773458454161,
         9.99999999999999822, 1.12829414671605333e-17, 1.2026194599794709e-17, 2.50706276000769697e-15, 2.24624122987970677e-15, 
         0.370764847417013987, 0.278456024942955571, 0.523838487672241171,0.388620152510322781, 0.298156764974678579]
y_opt = 32.6555929502463

def G19(X):
    b = np.array([-40,-2,-0.25,-4,-4,-1,-40,-60,5,1])
    e = np.array([-15,-27,-36,-18,-12])
    c = np.array([[30, -20, -10, 32, -10],
          [-20, 39, -6, -31, 32],
          [-10, -6, 10, -6, -10],
          [32, -31, -6, 39, -20],
          [-10, 32, -10, -20, 30]
         ])
    
    d = np.array([4, 8, 10, 6, 2])
    a = np.array([[-16, 2, 0, 1, 0],
          [0, -2, 0, 0.4, 2],
          [-3.5, 0, 2, 0, 0],
          [0, -2, 0, -4, -1],
          [0, -9, -2, 1, -2.8],
          [2, 0, -4, 0, 0],
          [-1, -1, -1, -1, -1],
          [-1, -2, -3, -2, -1],
          [1, 2, 3, 4, 5],
          [1, 1, 1, 1, 1]
         ])
    
    y = 0.0
    for i in range(5):
        for j in range(5):
            y = y+c[i,j]*X[10+i]*X[10+j]
    for j in range(5):
        y = y+2*d[j]*X[10+j]**3
    for i in range(10):
        y = y-b[i]*X[i]
    
    g1 = 0.0
    g2 = 0.0
    g3 = 0.0
    g4 = 0.0
    g5 = 0.0
    for i in range(5):
        g1 = g1-2*c[i,0]*X[10+i]
    g1 = g1-3*d[0]*X[10+0]**2-e[0]
    for i in range(10):
        g1 = g1+a[i,0]*X[i]
        
    for i in range(5):
        g2 = g2-2*c[i,1]*X[10+i]
    g2 = g2-3*d[1]*X[10+1]**2-e[1]
    for i in range(10):
        g2 = g2+a[i,1]*X[i]
    
    for i in range(5):
        g3 = g3-2*c[i,2]*X[10+i]
    g3 = g3-3*d[2]*X[10+2]**2-e[2]
    for i in range(10):
        g3 = g3+a[i,2]*X[i]
        
    for i in range(5):
        g4 = g4-2*c[i,3]*X[10+i]
    g4 = g4-3*d[3]*X[10+3]**2-e[3]
    for i in range(10):
        g4 = g4+a[i,3]*X[i]
    
    for i in range(5):
        g5 = g5-2*c[i,4]*X[10+i]
    g5 = g5-3*d[4]*X[10+4]**2-e[4]
    for i in range(10):
        g5 = g5+a[i,4]*X[i]
     
    return y,g1,g2,g3,g4,g5

In [4]:
# y = lambda X: G19(X)[0]
# g1 = lambda X: G19(X)[1]
# g2 = lambda X: G19(X)[2]
# g3 = lambda X: G19(X)[3]
# g4 = lambda X: G19(X)[4]
# g5 = lambda X: G19(X)[5]
# x0 = [0 for i in range(15)]
# # x0 = [5 for i in range(15)]
# # x0 = [10 for i in range(15)]

In [5]:
# maxfunction(g5,x0,bounds)

In [6]:
# 855.0+1216.999999624288+736.0+1231.9999997502223+892.0

In [7]:
### 适应度函数
def fitnessG19_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1220.174
    fmin = 0.008509619375000003
    count = np.sum(np.array(problem1(X))[1:5]>0)
    
    return -(problem1(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG19_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 66512.5
    fmin = -59.99999992891392
    vmax = 4931.99999937451
    e = 0.1 
    n = 5 
    
    res = G19(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:6]>0)
    
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness  


In [8]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10.0e+28)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10.0e+28)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG19_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/200))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG19_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/200))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [10]:
PSO=PSO_fit(pN=8000,dim=15,bounds=bounds)

In [ ]:
PSO.init_Population()

In [11]:
PSO.iterator()

[-5653.43016275 -9128.73738284 -2722.1097267  ... -8926.89819604
 -3042.46017019 -4854.51156332]
[-4145.43651037 -5422.30169817 -2722.1097267  ... -5760.92141486
 -3042.46017019 -1229.62836479]
[-2395.57056088 -3969.71749081 -2722.1097267  ... -5760.92141486
 -3042.46017019 -1131.75039556]
[-2395.57056088 -1913.80566165 -2722.1097267  ... -3713.10284006
 -2999.53256145 -1131.75039556]
[-1480.55318325 -1913.80566165 -1313.21544387 ... -1521.48148097
 -1300.42217609 -1131.75039556]
[-1184.40375124 -1058.10719254 -1043.30631197 ... -1107.85164086
 -1300.42217609  -658.53598655]
[-1089.74001608  -898.98298031  -586.33124412 ... -1038.40004728
  -948.47326065  -658.53598655]
[-697.96961639 -616.987399   -586.33124412 ... -855.95321176 -466.52492247
 -658.53598655]
[-697.96961639 -407.74277172 -491.97342109 ... -495.06198757 -417.58600414
 -572.80386138]
[-231.42243019 -407.74277172 -491.02078265 ... -495.06198757 -223.28086221
 -294.43896411]
[-231.42243019 -327.69126912 -387.46751275 ... -

[-34.86237563 -35.37840497 -34.64302923 ... -34.98177637 -34.86901762
 -34.90294078]
[-34.86237563 -34.82134489 -34.64302923 ... -34.98177637 -34.82996939
 -34.89345797]
[-34.86237563 -34.82134489 -34.64302923 ... -34.93624346 -34.70660532
 -34.72421507]
[-34.86237563 -34.82134489 -34.64302923 ... -34.93624346 -34.70660532
 -34.72421507]
[-34.86237563 -34.82134489 -34.64302923 ... -34.93624346 -34.70660532
 -34.72421507]
[-34.86237563 -34.82134489 -34.64302923 ... -34.80300087 -34.70660532
 -34.72421507]
[-34.86237563 -34.82134489 -34.64302923 ... -34.80300087 -34.70660532
 -34.72421507]
[-34.86237563 -34.82134489 -34.64302923 ... -34.80300087 -34.70660532
 -34.72421507]
[-34.86237563 -34.82134489 -34.64302923 ... -34.80300087 -34.70660532
 -34.72421507]
[-34.86237563 -34.82134489 -34.64302923 ... -34.80300087 -34.70660532
 -34.72421507]
[-34.86237563 -34.75204584 -34.34057851 ... -34.73032124 -34.70660532
 -34.72421507]
[-34.86237563 -34.28409615 -34.34057851 ... -34.73032124 -34.2630

[-33.28352147 -33.28258338 -33.27323293 ... -33.24909425 -33.28229558
 -33.27343511]
[-33.28352147 -33.28258338 -33.27323293 ... -33.24909425 -33.2717389
 -33.27343511]
[-33.28352147 -33.28258338 -33.26516382 ... -33.24909425 -33.2717389
 -33.27343511]
[-33.26444083 -33.28258338 -33.26516382 ... -33.24909425 -33.2717389
 -33.27343511]
[-33.26444083 -33.26598101 -33.26516382 ... -33.24909425 -33.2717389
 -33.26058926]
[-33.26444083 -33.26598101 -33.24473546 ... -33.24909425 -33.21748254
 -33.25698568]
[-33.23733114 -33.22816231 -33.24473546 ... -33.23500082 -33.21748254
 -33.22571665]
[-33.22208747 -33.22816231 -33.24473546 ... -33.23500082 -33.21748254
 -33.22571665]
[-33.22208747 -33.22816231 -33.20639049 ... -33.22177893 -33.21748254
 -33.22571665]
[-33.21043605 -33.22816231 -33.20639049 ... -33.22177893 -33.21748254
 -33.22571665]
[-33.21043605 -33.22816231 -33.20639049 ... -33.22177893 -33.21748254
 -33.22571665]
[-33.21043605 -33.2089371  -33.20639049 ... -33.22177893 -33.21748254

[-33.1563241  -33.15636474 -33.1563241  ... -33.15642271 -33.15631775
 -33.15637092]
[-33.1563241  -33.156357   -33.1563241  ... -33.15632767 -33.15631775
 -33.15637092]
[-33.1563241  -33.15633399 -33.1563241  ... -33.15632767 -33.15631775
 -33.15637092]
[-33.1563241  -33.15633399 -33.1563241  ... -33.15632767 -33.15631775
 -33.15629154]
[-33.15628895 -33.15633399 -33.15625591 ... -33.15632767 -33.15628062
 -33.15629154]
[-33.15628895 -33.15627388 -33.15625591 ... -33.15632767 -33.15628062
 -33.15629154]
[-33.15625582 -33.15627388 -33.15625591 ... -33.15632767 -33.15628062
 -33.15629154]
[-33.15625582 -33.15627388 -33.15625591 ... -33.15631014 -33.15624465
 -33.15629154]
[-33.15625582 -33.15626786 -33.15625591 ... -33.15626549 -33.15624465
 -33.15626163]
[-33.15625582 -33.15626786 -33.15625591 ... -33.1562629  -33.15623356
 -33.15626163]
[-33.15625582 -33.15622382 -33.15625591 ... -33.1562629  -33.15623356
 -33.15626163]
[-33.15625582 -33.15622382 -33.15625591 ... -33.1562629  -33.1562

[-33.15610526 -33.15610538 -33.15610535 ... -33.15610533 -33.15610533
 -33.15610541]
[-33.15610526 -33.15610538 -33.15610523 ... -33.15610533 -33.15610526
 -33.15610533]
[-33.15610526 -33.15610538 -33.15610523 ... -33.15610533 -33.15610526
 -33.15610533]
[-33.15610525 -33.15610538 -33.15610523 ... -33.15610526 -33.15610526
 -33.15610533]
[-33.15610521 -33.15610534 -33.15610523 ... -33.15610526 -33.15610525
 -33.15610533]
[-33.15610521 -33.15610518 -33.15610522 ... -33.15610526 -33.15610525
 -33.15610533]
[-33.15610521 -33.15610518 -33.15610522 ... -33.15610526 -33.15610525
 -33.15610533]
[-33.15610518 -33.15610518 -33.15610522 ... -33.15610522 -33.15610525
 -33.15610533]
[-33.15610518 -33.15610518 -33.15610522 ... -33.15610518 -33.1561052
 -33.15610511]
[-33.15610518 -33.15610518 -33.15610522 ... -33.15610511 -33.15610512
 -33.15610511]
[-33.15610517 -33.15610518 -33.15610517 ... -33.15610511 -33.15610512
 -33.15610511]
[-33.15610517 -33.15610517 -33.15610517 ... -33.15610511 -33.15610

...

[-32.65559296 -32.65559296 -32.65559296 ... -32.65559296 -32.65559296
 -32.65559296]
[-32.65559296 -32.65559296 -32.65559296 ... -32.65559296 -32.65559296
 -32.65559296]
[-32.65559296 -32.65559296 -32.65559296 ... -32.65559296 -32.65559296
 -32.65559296]
[-32.65559296 -32.65559296 -32.65559296 ... -32.65559296 -32.65559296
 -32.65559296]
[-32.65559296 -32.65559296 -32.65559296 ... -32.65559296 -32.65559296
 -32.65559296]
[-32.65559296 -32.65559296 -32.65559295 ... -32.65559296 -32.65559296
 -32.65559296]
[-32.65559296 -32.65559296 -32.65559295 ... -32.65559296 -32.65559296
 -32.65559296]
[-32.65559296 -32.65559296 -32.65559295 ... -32.65559296 -32.65559296
 -32.65559296]
[-32.65559296 -32.65559296 -32.65559295 ... -32.65559296 -32.65559295
 -32.65559296]
[-32.65559296 -32.65559295 -32.65559295 ... -32.65559296 -32.65559295
 -32.65559296]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559296 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559296 -32.6555

[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.6555

[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.6555

[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.65559295
 -32.65559295]
[-32.65559295 -32.65559295 -32.65559295 ... -32.65559295 -32.6555

(array([[1.53409500e-24, 4.67234736e-17, 3.94599038e+00, ...,
         5.23838485e-01, 3.88620144e-01, 2.98156762e-01],
        [3.93570781e-19, 5.89624201e-17, 3.94599038e+00, ...,
         5.23838484e-01, 3.88620143e-01, 2.98156763e-01],
        [0.00000000e+00, 0.00000000e+00, 3.94599038e+00, ...,
         5.23838485e-01, 3.88620143e-01, 2.98156763e-01],
        ...,
        [0.00000000e+00, 0.00000000e+00, 3.94599038e+00, ...,
         5.23838485e-01, 3.88620144e-01, 2.98156762e-01],
        [0.00000000e+00, 2.67507322e-18, 3.94599038e+00, ...,
         5.23838485e-01, 3.88620143e-01, 2.98156763e-01],
        [0.00000000e+00, 0.00000000e+00, 3.94599038e+00, ...,
         5.23838485e-01, 3.88620143e-01, 2.98156763e-01]]),
 array([-32.65559295, -32.65559295, -32.65559295, ..., -32.65559295,
        -32.65559295, -32.65559295]))